In [106]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import glob
import re
import numpy as np

Собираем данные по ключевой ставке

In [179]:
url = 'https://www.cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From=17.09.2013&UniDbQuery.To=02.06.2023'
response = requests.get(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
data = []
for row in table.find_all('tr'):
    cols = [col.text.strip() for col in row.find_all('td')]
    data.append(cols)
df_key_rate = pd.DataFrame(data)
df_key_rate.columns = ['date', 'value']
df_key_rate.dropna(inplace=True)
df_key_rate['value'] = df_key_rate['value'].str.replace(',', '.')
df_key_rate.head()

,date,value
1,02.06.2023,7.50
2,01.06.2023,7.50
3,31.05.2023,7.50
4,30.05.2023,7.50
5,29.05.2023,7.50


In [27]:
df_key_rate.to_csv('df_key_rate.csv', index=False)

Депозиты и кредиты

In [219]:
#разбиваем листы эксель на отдельные файлы

to_df_info = glob.glob('/Users/svetlanamaslennikova/Downloads/deposits/stat_bn_*')
q = pd.DataFrame(to_df_info)[0]

# Загрузка файла Excel
for i in range(0, len(to_df_info)):
    excel_file = pd.ExcelFile(q[i])

# Обход всех листов в файле и сохранение их в отдельные файлы
    for sheet_name in excel_file.sheet_names:
    # Чтение данных из листа
        df = excel_file.parse(sheet_name)
        
    # Создание нового файла с именем, соответствующим имени листа
        digits = re.findall(r'\d+', q[i])
        new_file_name = sheet_name + f"_{digits[0]}" + '.xlsx'
        df.to_excel(new_file_name, index=False)

In [270]:
#собираем листы разных годов в одну таблицу

#Объем привлеченных кредитными организациями вкладов (депозитов) физических лиц  
to_df_info = glob.glob('/Users/svetlanamaslennikova/2_1*')
q = pd.DataFrame(to_df_info)[0]


month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

columns = ['trash', 'metric']
for i in np.arange(0,12):
        columns.append(f"01.{month[i]}")

string = q[0]
year = re.search(r'\d{4}(?=\.)', string).group()

df = pd.read_excel(q[0])
df.dropna(inplace=True)
df.columns = columns
df = df.drop(['trash'], axis=1)

filtered_df = df[df['metric'].str.contains('\*')] #удаляем агрегаты
df = df.drop(filtered_df.index)
df = pd.melt(df, id_vars=['metric'], var_name='date', value_name='value')
df['date'] = df['date'] + '.' + year

        
for i in range(1, len(to_df_info)):
    
    string = q[i]
    year = re.search(r'\d{4}(?=\.)', string).group()

    data =  pd.read_excel(q[i])
    data.dropna(inplace=True)
    data.columns = columns
    data = data.drop(['trash'], axis=1)

    filtered_df = data[data['metric'].str.contains('\*')] #удаляем агрегаты
    data = data.drop(filtered_df.index)
    data = pd.melt(data, id_vars=['metric'], var_name='date', value_name='value')
    data['date'] = data['date'] + '.' + year
    info = pd.concat([df, data])
    df = info
    
df['metric'] = df['metric'].str.strip()
df = df[~df['metric'].str.contains('в иностранной валюте')]
df = df[~df['metric'].str.contains('в рублях')]
df = df[~df['metric'].str.contains('депозиты')]


deposits_people = df[['date', 'value']]
deposits_people = deposits_people.groupby(deposits_people['date']).sum()
deposits_people.head(2)

,value
date,
01.01.2019,2.846023e+07
01.01.2020,3.054901e+07


In [357]:
deposits_people.to_csv('deposits_people.csv', index=False)

In [272]:
#собираем листы разных годов в одну таблицу

#Объем привлеченных кредитными организациями средств юридических лиц
to_df_info = glob.glob('/Users/svetlanamaslennikova/2_2*')
q = pd.DataFrame(to_df_info)[0]


month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

columns = ['trash', 'metric']
for i in np.arange(0,12):
        columns.append(f"01.{month[i]}")

string = q[0]
year = re.search(r'\d{4}(?=\.)', string).group()

df = pd.read_excel(q[0])
df.dropna(inplace=True)
df.columns = columns
df = df.drop(['trash'], axis=1)

filtered_df = df[df['metric'].str.contains('\*')] #удаляем агрегаты
df = df.drop(filtered_df.index)
df = pd.melt(df, id_vars=['metric'], var_name='date', value_name='value')
df['date'] = df['date'] + '.' + year

        
for i in range(1, len(to_df_info)):
    
    string = q[i]
    year = re.search(r'\d{4}(?=\.)', string).group()

    data =  pd.read_excel(q[i])
    data.dropna(inplace=True)
    data.columns = columns
    data = data.drop(['trash'], axis=1)

    
    data = pd.melt(data, id_vars=['metric'], var_name='date', value_name='value')
    data['date'] = data['date'] + '.' + year
    info = pd.concat([df, data])
    df = info
    
df['metric'] = df['metric'].str.strip()
filtered_df = df[df['metric'].str.contains('\*')] #удаляем агрегаты
df = df.drop(filtered_df.index)    
df = df[~df['metric'].str.contains('в рублях')]
df = df[~df['metric'].str.contains('в иностранной валюте')]
df = df[~df['metric'].str.contains('Депозиты и прочие привлеченные  средства юридических лиц2  - всего')]
df = df[~df['metric'].str.contains('по срокам привлечения:')]
df = df[~df['metric'].str.contains('Справочно: Депозиты индивидуальных предпринимателей')]
df = df[~df['metric'].str.contains('Кредиты, депозиты и прочие  привлеченные средства кредитных организаций - всего')]
df = df[~df['metric'].str.contains('Депозиты и прочие привлеченные  средства юридических лиц  - всего')]


deposits_firms = df[['date', 'value']]
deposits_firms = deposits_firms.groupby(deposits_firms['date']).sum()
deposits_firms.head(2)

,value
date,
01.01.2019,2.165197e+07
01.01.2020,2.168399e+07


In [358]:
deposits_firms.to_csv('deposits_firms.csv', index=False)

In [273]:
#собираем листы разных годов в одну таблицу

#кредиты фирмам
to_df_info = glob.glob('/Users/svetlanamaslennikova/3_1*')
q = pd.DataFrame(to_df_info)[0]

month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

columns = ['trash', 'metric']
for i in np.arange(0,12):
        columns.append(f"01.{month[i]}")

string = q[0]
year = re.search(r'\d{4}(?=\.)', string).group()

df = pd.read_excel(q[0])
df.dropna(inplace=True)
df.columns = columns
df = df.drop(['trash'], axis=1)

filtered_df = df[df['metric'].str.contains('\*')] #удаляем агрегаты
df = df.drop(filtered_df.index)
df = pd.melt(df, id_vars=['metric'], var_name='date', value_name='value')
df['date'] = df['date'] + '.' + year

        
for i in range(1, len(to_df_info)):
    
    string = q[i]
    year = re.search(r'\d{4}(?=\.)', string).group()

    data =  pd.read_excel(q[i])
    data.dropna(inplace=True)
    data.columns = columns
    data = data.drop(['trash'], axis=1)

    filtered_df = data[data['metric'].str.contains('\*')] #удаляем агрегаты
    data = data.drop(filtered_df.index)
    data = pd.melt(data, id_vars=['metric'], var_name='date', value_name='value')
    data['date'] = data['date'] + '.' + year
    info = pd.concat([df, data])
    df = info
    
df['metric'] = df['metric'].str.strip()
df = df[~df['metric'].str.contains('кредиты и прочие средства,\n     предоставленные корпоративным клиентам.')]
df = df[~df['metric'].str.contains('до ')]
df = df[~df['metric'].str.contains('от ')]
df = df[~df['metric'].str.contains('свыше')]
df = df[~df['metric'].str.contains('депозиты')]
df = df[~df['metric'].str.contains('физическим ')]

#df = df[~df['metric'].str.contains('депозиты')]


credits_firms = df[['date', 'value']]
credits_firms = credits_firms.groupby(credits_firms['date']).sum()
credits_firms.head(2)

,value
date,
01.01.2019,3.801098e+07
01.01.2021,1.162363e+07


In [359]:
credits_firms.to_csv('credits_firms.csv', index=False)

In [275]:
#Кредиты населению
excel_file = pd.ExcelFile('/Users/svetlanamaslennikova/Downloads/02_05_Debt_ind.xlsx')

# Обход всех листов в файле и сохранение их в отдельные файлы
for sheet_name in excel_file.sheet_names:
    # Чтение данных из листа
    df = excel_file.parse(sheet_name)
        
    # Создание нового файла с именем, соответствующим имени листа
    new_file_name = 'credits_people_' + sheet_name + '.xlsx'
    df.to_excel(new_file_name, index=False)

/usr/local/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [282]:
df = pd.read_excel('/Users/svetlanamaslennikova/credits_people_итого.xlsx', header=1)
df = df[df['Unnamed: 0'].str.contains('РОССИЙСКАЯ ФЕДЕРАЦИЯ')]
df.dropna(inplace=True)

df = pd.melt(df, id_vars=['Unnamed: 0'], var_name='date', value_name='value')
credits_people = df[['date', 'value']]
credits_people.head()

,date,value
0,01.02.2019,15005779
1,01.03.2019,15190019
2,01.04.2019,15449315
3,01.05.2019,15761207
4,01.06.2019,16014846


In [360]:
credits_people.to_csv('credits_people.csv', index=False)

In [371]:
df = pd.read_excel('/Users/svetlanamaslennikova/credits_people_в т.ч.просроч..xlsx', header=1)
df = df[df['Unnamed: 0'].str.contains('РОССИЙСКАЯ ФЕДЕРАЦИЯ')]
df.dropna(inplace=True)

df = pd.melt(df, id_vars=['Unnamed: 0'], var_name='date', value_name='value')
overdue_credits_people = df[['date', 'value']]
overdue_credits_people.head()

,date,value
0,01.02.2019,774230
1,01.03.2019,765450
2,01.04.2019,754898
3,01.05.2019,761902
4,01.06.2019,769890


In [361]:
overdue_credits_people.to_csv('overdue_credits_people.csv', index=False)

In [365]:
#прибыль
df = pd.read_excel('/Users/svetlanamaslennikova/Downloads/прибыль.xlsx')
profit_fin_org = df
profit_fin_org['date'] = profit_fin_org['date'].dt.strftime('%d.%m.%Y')
profit_fin_org.head()

,date,value
0,01.01.2021,204 735
1,01.02.2021,372 998
2,01.03.2021,577 636
3,01.04.2021,778 661
4,01.05.2021,994 646


In [367]:
profit_fin_org.to_csv('overdue_credits_people.csv', index=False)

In [334]:
#Динамика максимальной процентной ставки 
#(по вкладам в российских рублях) 
#десяти кредитных организаций, привлекающих наибольший объём депозитов физических лиц

url = 'https://www.cbr.ru/statistics/avgprocstav/?UniDbQuery.Posted=True&UniDbQuery.From=2.07.2009&UniDbQuery.To=3.05.2023'
response = requests.get(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
data = []
for row in table.find_all('tr'):
    cols = [col.text.strip() for col in row.find_all('td')]
    data.append(cols)
max_deposit_rate = pd.DataFrame(data)
max_deposit_rate.columns = ['date', 'value']
max_deposit_rate.dropna(inplace=True)

max_deposit_rate['date'] =  '01.' + max_deposit_rate['date'].str.extract('\.(.+)\.') + '.' + max_deposit_rate['date'].str.extract('(\d{4})') 
max_deposit_rate['value'] = max_deposit_rate['value'].str.replace(',', '.')
max_deposit_rate['value'] = max_deposit_rate['value'].astype(float)

max_deposit_rate = max_deposit_rate.groupby('date').mean()

max_deposit_rate.head()

,value
date,
01.01.2010,12.430000
01.01.2011,8.176667
01.01.2012,9.498333
01.01.2013,9.841667
01.01.2014,8.310000


In [368]:
max_deposit_rate.to_csv('max_deposit_rate.csv', index=False)

In [356]:
#Структурный дефицит/профицит ликвидности банковского сектора

url = 'https://www.cbr.ru/hd_base/bliquidity/?UniDbQuery.Posted=True&UniDbQuery.From=01.05.2017&UniDbQuery.To=02.06.2023'
response = requests.get(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')
data = []
for row in table.find_all('tr'):
    cols = [col.text.strip() for col in row.find_all('td')]
    data.append(cols)
bank_luquidity = pd.DataFrame(data)
bank_luquidity.dropna(inplace=True)
columns = ['date', 'value']
bred = list(range(0,10))
for i in bred:
    columns.append(bred[i])

bank_luquidity.columns = columns

bank_luquidity = bank_luquidity[['date', 'value']]
bank_luquidity['value'] = bank_luquidity['value'].str.replace(' ', '')
bank_luquidity['value'] = bank_luquidity['value'].str.replace(',', '.')
bank_luquidity['value'] = bank_luquidity['value'].astype(float)
bank_luquidity = bank_luquidity.groupby('date').mean()

bank_luquidity.head()


,value
date,
01.01.2018,-2639.2
01.01.2019,-3015.9
01.01.2020,-2761.1
01.01.2021,-203.7
01.01.2022,-1691.1


In [369]:
bank_luquidity.to_csv('bank_luquidity.csv', index=False)